# Classifying requirements 

We would be using **Keras** and **Tensorflow** for creating the classification model. Hence, let's install these libraries first. 

In [ ]:
!pip install keras
!pip install tensorflow

Importing different libraries that we would be using for creating the model. 

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np

Importing the .csv file with the requirements and their corresponding labels. 

In [3]:
df = pd.read_csv("requirements.csv")
df.head()

requirement  label
0  The Crew Oxygen hoses shall not be adversely a...      0
1  The slide/raft shall be capable of withstandin...      0
2  The Secondary Lock Actuator shall be designed ...      0
3  The main landing gear shall operate within the...      0
4  The HCM shall meet performance requirements af...      0

Let's check the number of requirements and the types of requirements in our dataset. 

In [4]:
df.shape

(253, 2)

In [5]:
# Uncomments this cell to convert the labels into string type
# df.label = df.label.astype(str)
# df.label.unique()

Environmental requirements are labeled 0. <br> 
Suitability requirements are labeled 1. <br>
Design requirements are labeled as 2. <br> 

The next step is to split the requirements into training and test sets. Since, we are dealing with an unbalanced dataset (having unequal number of each type of requirements), we need to be careful which doing the split - need for **stratified sampling**. This is done to make sure that the distribution of the training and test sets are similar. We will use 75% of the requirements for training and 25% for testing. <br> 

Next step would be to convert our textual requirements into numbers --> tokenization. For this exercise, we will choose a 5000 word vocabulary based on the training set `.fit_on_texts`. This is followed by converting the requirements into sequences of numbers (there is a specific number assigned to each token in the previous step). <br> 

Lastly, we will post pad the sequences with zeroes to make sure that all the input sequences are of the same length. 

In [6]:
requirements = df['requirement'].values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(requirements, y, stratify = y, test_size = 0.25, random_state = 42)

#For printing the requirements in the end 
print_req = X_test

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(X_train)  # Only fit the train data

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test) # There will be some unseen words here since the tokenizer only took into account the training set 

vocab_size = len(tokenizer.word_index)+1 # Adding 1 because of reserve 0 index

# Padding sequences so that all of them has the same length
maxlen = 100 

# Padding sequences to make sure that each of them have same size --> post padding --> Adds zeroes to the end of the sentences 
# till they reach a length of 100
X_train = pad_sequences(X_train, padding = "post", maxlen = maxlen)
X_test = pad_sequences(X_test, padding = "post", maxlen = maxlen)

# np.unique(y_train, return_counts = True) # Counts the number of examples of each type in the test set

## GloVe Pre-trained word embeddings

Global vectors for word representation (GloVe) developed by Stanford NLP Group to obtain word embeddings. <br> 
GloVe is an unsupervised learning algorithm for obtaining vector representations for words. Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.


## Creating Bidirectional LSTM model for classification of requirements

LSTM in its core, preserves information from inputs that has already passed through it using the hidden state. Unidirectional LSTM only preserves information of the past because the only inputs it has seen are from the past. <br> 

Using **bidirectional** LSTM will run your inputs in two ways, one from past to future and one from future to past and what differs this approach from unidirectional is that in the LSTM that runs backwards you preserve information from the future and using the two hidden states combined you are able in any point in time to preserve information from both past and future.

What they are suited for is a very complicated question but BiLSTMs show very good results as they can understand context better. 

In [7]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

# Call back will stop training when a monitored metric has stopped improving 
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 3, mode = 'auto', min_delta = 0.001)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)), 
    tf.keras.layers.Dense(embedding_dim, activation = 'relu'), 
    tf.keras.layers.Dense(3, activation = 'softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         81700     
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 100)               20100     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 262,903
Trainable params: 262,903
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(loss = 'sparse_categorical_crossentropy', 
             optimizer = 'adam', 
             metrics = ['accuracy'])

history = model.fit(X_train, y_train,
                   epochs = 30,
                   validation_data = (X_test, y_test), callbacks = [callback])

Epoch 1/30
6/6 [==============================] - 2s 137ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 2.1803 - val_accuracy: 0.7656
Epoch 2/30
6/6 [==============================] - 0s 59ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 1.5647 - val_accuracy: 0.7500
Epoch 3/30
6/6 [==============================] - 0s 51ms/step - loss: 4.1400e-04 - accuracy: 1.0000 - val_loss: 1.1509 - val_accuracy: 0.7812
Epoch 4/30
6/6 [==============================] - 0s 52ms/step - loss: 2.9157e-04 - accuracy: 1.0000 - val_loss: 1.0457 - val_accuracy: 0.8281
Epoch 5/30
6/6 [==============================] - 0s 52ms/step - loss: 2.1288e-04 - accuracy: 1.0000 - val_loss: 1.0378 - val_accuracy: 0.8438
Epoch 6/30
6/6 [==============================] - 0s 52ms/step - loss: 1.1793e-04 - accuracy: 1.0000 - val_loss: 1.0754 - val_accuracy: 0.8281
Epoch 7/30
6/6 [==============================] - 0s 52ms/step - loss: 6.5211e-05 - accuracy: 1.0000 - val_loss: 1.1359 - val_accuracy: 0.8125
Epoch 

We were able to achieve a training accuracy of 100% and a validation accuracy of **81.25%**.  <br> 
It is important to keep in mind the concerns regarding overfitting when the model accuracy is very high - this can happen due to a small training set and training for higher number of epochs. Overfitted models tend NOT to generalize well when faced with new data. <br> 
Since, our validation accuracy is also pretty high, we can disregard the overfitting concerns for now (might change as we see more/different kind of requirements.) 

## Testing requirements (y_test)

Environmental requirements are labeled 0. <br> 
Suitability requirements are labeled 1. <br>
Design requirements are labeled as 2. <br> 

In [26]:
predictions = model.predict(X_test)
pred_prob = np.round(predictions,3)

for i,x in enumerate(predictions):
    k = x.argmax(axis = 0)
    actual_type = y_test[i]
    print_req_1 = print_req[i]
    pred_prob_1 = pred_prob[i]
    print(f'Probabilities: {pred_prob_1}; Predicted type: {k}; Actual type: {actual_type}; Text: {print_req_1} \n')

Probabilities: [0.999 0.001 0.   ]; Predicted type: 0; Actual type: 0; Text: The Secondary Lock Actuator shall be designed to withstand the airplane sonic environment. 

Probabilities: [0.001 0.003 0.996]; Predicted type: 2; Actual type: 2; Text: The REU shall use the database load version to enable the appropriate mapping logic group. 

Probabilities: [1. 0. 0.]; Predicted type: 0; Actual type: 0; Text: On the new airplane, the WAP shall meet all performance requirements during and after experiencing Temperature in accordance with D048W400-01. 

Probabilities: [0.602 0.187 0.211]; Predicted type: 0; Actual type: 2; Text: The pump shall include features to ensure hydraulic cavitation does not occur. 

Probabilities: [1. 0. 0.]; Predicted type: 0; Actual type: 0; Text: All HSTAs shall be designed to withstand continuous exposure to temperature extremes of -3.1 meters to 3.1 meters with no permanent damage. 

Probabilities: [0.002 0.142 0.856]; Predicted type: 2; Actual type: 2; Text: Th